<a href="https://colab.research.google.com/github/jinyoung0711/iwantsuccess/blob/main/tokenized_AI_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt
import urllib3
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data= pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/실습용 자료/1. 실습용자료.txt', sep ='|',encoding='CP949', header=0)
submission= pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/모델 개발용 자료/2. 모델개발용자료.txt', sep ='|',encoding='CP949', header=0)
data

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스


In [ ]:
data.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육


In [ ]:
submission.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"


In [ ]:
ks = pd.read_csv('/content/drive/MyDrive/통계청_AI경진대회/한국표준산업분류(10차)_국문_test.csv',encoding='CP949',header=2)

In [ ]:
#input2 세부항목명으로 들어갈 경우 input2 : digit_3까지.
#ks['코드.4']=ks['코드.4']//100
input2 = ks[['코드.2','항목명.3']].dropna()
input2 = input2.reset_index(drop=True)
input2['코드.2']=input2['코드.2'].astype(int)
input2['항목명.3']=input2['항목명.3'].str.rstrip('업')
input2.rename(columns={'코드.2':'digit_3'},inplace=True)
input2.rename(columns={'항목명.3':'NLP'},inplace=True)
input2.head()

,digit_3,NLP
0,11,곡물 및 기타 식량작물 재배
1,12,소 사육
2,13,작물재배 및 축산 복합농
3,14,작물재배 관련 서비스
4,15,수렵 및 관련 서비스


In [ ]:
#ks['코드.3']=ks['코드.3']//10

In [ ]:
#input 세부항목명으로 들어갈 경우
input3 = ks[['코드.3','항목명.3']].dropna()
input3 = input3.reset_index(drop=True)
input3['코드.3']=input3['코드.3'].astype(int)
#input3['항목명.3']=input3['항목명.3'].str.rstrip('업')
input3.rename(columns={'코드.3':'digit_3'},inplace=True)
input3.rename(columns={'항목명.3':'NLP'},inplace=True)
input3

,digit_3,NLP
0,111,곡물 및 기타 식량작물 재배업
1,112,"채소, 화훼작물 및 종묘 재배업"
2,113,"과실, 음료용 및 향신용 작물 재배업"
3,114,기타 작물 재배업
4,115,시설작물 재배업
...,...,...
490,9699,그 외 기타 분류 안된 개인 서비스업
491,9700,가구 내 고용활동
492,9810,자가 소비를 위한 가사 생산 활동
493,9820,자가 소비를 위한 가사 서비스 활동


In [ ]:
input3['NLP'] = input3['NLP'].str.rstrip('업')

In [ ]:
input3['NLP']

0          곡물 및 기타 식량작물 재배
1         채소, 화훼작물 및 종묘 재배
2      과실, 음료용 및 향신용 작물 재배
3                 기타 작물 재배
4                  시설작물 재배
              ...         
490    그 외 기타 분류 안된 개인 서비스
491              가구 내 고용활동
492     자가 소비를 위한 가사 생산 활동
493    자가 소비를 위한 가사 서비스 활동
494              국제 및 외국기관
Name: NLP, Length: 495, dtype: object

In [ ]:
a1=[]
for i in range(len(input2)):
  a1.append('id_'+str(i+1000001))
a2=[]
for i in range(len(input3)):
  a2.append('id_'+str(i+1000001))

In [ ]:
import pandas as pd
input2.insert(0,'AI_id',a1)
input3.insert(0,'AI_id',a2)

input들 정리 끝

In [ ]:
col = ['text_obj','text_mthd','text_deal']
NLP = data[col].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
NLP.head()

0          카센터에서 자동차부분정비 타이어오일교환
1        상점내에서 일반인을 대상으로 채소.과일판매
2    절단하여사업체에도매 공업용고무를가지고 합성고무도매
3           영업점에서 일반소비자에게 열쇠잠금장치
4       어린이집 보호자의 위탁을 받아 취학전아동보육
dtype: object

In [ ]:
df = data[['AI_id','digit_3']].copy()
df['NLP'] = NLP

In [ ]:
df_concat1=pd.concat([df,input2])
df_concat2=pd.concat([df,input3])

In [ ]:
df_concat1['NLP']=df_concat1['NLP'].astype(str)
df_concat2['NLP']=df_concat2['NLP'].astype(str)
df_concat1.reset_index(inplace=True)
df_concat2.reset_index(inplace=True)

In [ ]:
NLP1 = df_concat1['NLP']
NLP2 = df_concat2['NLP']

In [ ]:
NLP1[1]

'상점내에서 일반인을 대상으로 채소.과일판매'

In [ ]:
# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거합니다.
import re
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

In [ ]:
for i in NLP1:
  text_cleaning(i)
for i in NLP2:
  text_cleaning(i)

In [ ]:
accessKey = "f1f676f2-f880-4604-9d4d-d5245f2ecaba"
analysisCode = "morp"
text = NLP1[1]


'상점내에서 일반인을 대상으로 채소.과일판매'

In [ ]:
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 

In [ ]:
accessKey = "f1f676f2-f880-4604-9d4d-d5245f2ecaba"
analysisCode = "morp"
AI_Token1 = []

for i in range(len(NLP1)):
  text = NLP1[i]
  AI_Token1.append(text)

In [ ]:
AI_Token1

['카센터에서 자동차부분정비 타이어오일교환',
 '상점내에서 일반인을 대상으로 채소.과일판매',
 '절단하여사업체에도매 공업용고무를가지고 합성고무도매',
 '영업점에서 일반소비자에게 열쇠잠금장치',
 '어린이집 보호자의 위탁을 받아 취학전아동보육',
 '철 절삭.용접 카프라배관자재',
 '음식점에서 접객시설을 갖추고 참치회(일본식)',
 '쌀을 가지고 가공하여 떡제조',
 '시청에서 재정과인력 일반공공행정 지방행정 집행',
 '영업장에서 고객의뢰를 받아 내부전기공사',
 '멸치 입고, 가공 nan',
 '음식점 접객시설을 갖추고 냉면',
 '매장에서 일반인 대상으로 의류소매',
 '음식점에서 접객시설을 갖추고 회',
 '상점내 일반인을 대상으로 여러곡물소매판매',
 '태권도장에서 학생및일반대사으로 태권도',
 '사업장에서 고객요청으로 자동차세차업',
 '선박,화물 운송화물 검수',
 '매장에서 일반인에게 핸드폰소매',
 '영업장에서 일반 고객을 대상 자동차 임대업',
 '쌀을가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편판매',
 '당구 nan nan',
 '유흥주점에서 무도시설을 갖추고 술판매',
 '점포에서 일반인대상으로소매 건어물',
 '음료점에서 빙수, 아이스크림, 커피, 디저트 제공 커피.빙수등을제공',
 '영업점에서 일반소비자에게 오징어빵판매',
 '고무 절단가공 가스켓,다이아후레임',
 '사무실에서 직원들대상 교육관',
 '미용실에서 일반인대상 컷.파마',
 'Resim 재료구입, 세미앗세이 nan',
 '사업장에서 소비자를 대상으로 가정용가스 소매',
 '생지를 구입하여 가공업체가공의뢰하여 완제품수출 폴리에스테르.원단,직물',
 '음식점에서 접객시설을 갖추고 소갈비살',
 '세탁소에서 고객의 의뢰를 받아 가정용 세탁물 세탁',
 '음식점 접객시설을갖추고 치킨',
 '음식점에서 접객 시설을 갖추고 중화요리',
 '교회 기독교계통종교활동 종교서비스',
 '카페에서 접객시설을갖추고 커피,음료수',
 '가게에서 고객에게 잡화판매',
 '교회에서 기독

In [ ]:
accessKey = "f1f676f2-f880-4604-9d4d-d5245f2ecaba"
analysisCode = "morp"
AI_Token2 = []

for i in range(len(NLP2)):
  text2 = NLP2[i]
  AI_Token2.append(text2)

In [ ]:
AI_Token2

['카센터에서 자동차부분정비 타이어오일교환',
 '상점내에서 일반인을 대상으로 채소.과일판매',
 '절단하여사업체에도매 공업용고무를가지고 합성고무도매',
 '영업점에서 일반소비자에게 열쇠잠금장치',
 '어린이집 보호자의 위탁을 받아 취학전아동보육',
 '철 절삭.용접 카프라배관자재',
 '음식점에서 접객시설을 갖추고 참치회(일본식)',
 '쌀을 가지고 가공하여 떡제조',
 '시청에서 재정과인력 일반공공행정 지방행정 집행',
 '영업장에서 고객의뢰를 받아 내부전기공사',
 '멸치 입고, 가공 nan',
 '음식점 접객시설을 갖추고 냉면',
 '매장에서 일반인 대상으로 의류소매',
 '음식점에서 접객시설을 갖추고 회',
 '상점내 일반인을 대상으로 여러곡물소매판매',
 '태권도장에서 학생및일반대사으로 태권도',
 '사업장에서 고객요청으로 자동차세차업',
 '선박,화물 운송화물 검수',
 '매장에서 일반인에게 핸드폰소매',
 '영업장에서 일반 고객을 대상 자동차 임대업',
 '쌀을가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편판매',
 '당구 nan nan',
 '유흥주점에서 무도시설을 갖추고 술판매',
 '점포에서 일반인대상으로소매 건어물',
 '음료점에서 빙수, 아이스크림, 커피, 디저트 제공 커피.빙수등을제공',
 '영업점에서 일반소비자에게 오징어빵판매',
 '고무 절단가공 가스켓,다이아후레임',
 '사무실에서 직원들대상 교육관',
 '미용실에서 일반인대상 컷.파마',
 'Resim 재료구입, 세미앗세이 nan',
 '사업장에서 소비자를 대상으로 가정용가스 소매',
 '생지를 구입하여 가공업체가공의뢰하여 완제품수출 폴리에스테르.원단,직물',
 '음식점에서 접객시설을 갖추고 소갈비살',
 '세탁소에서 고객의 의뢰를 받아 가정용 세탁물 세탁',
 '음식점 접객시설을갖추고 치킨',
 '음식점에서 접객 시설을 갖추고 중화요리',
 '교회 기독교계통종교활동 종교서비스',
 '카페에서 접객시설을갖추고 커피,음료수',
 '가게에서 고객에게 잡화판매',
 '교회에서 기독

In [ ]:
import pickle
with open('/content/drive/MyDrive/통계청_AI경진대회/data/AI_Token1','wb') as f:
    pickle.dump(AI_Token1,f)

In [ ]:
import pickle
with open('/content/drive/MyDrive/통계청_AI경진대회/data/AI_Token2','wb') as f:
    pickle.dump(AI_Token2,f)